# Quick Start Guide to GenAIRR

Welcome to the Quick Start Guide for GenAIRR, a Python module designed for generating synthetic Adaptive Immune Receptor Repertoire (AIRR) sequences. This guide will walk you through the basic usage of GenAIRR, including setting up your environment, simulating heavy and light chain sequences, and customizing your simulations.


## Installation

Before you begin, ensure that you have Python 3.x installed on your system. GenAIRR can be installed using pip, Python's package installer. Execute the following command in your terminal:


In [ ]:
import pandas as pd
# Install GenAIRR using pip
#!pip install GenAIRR

## Setting Up

To start using GenAIRR, you need to import the necessary classes from the module. We'll also set up a `DataConfig` object to specify our configuration.


In [1]:
# Importing GenAIRR classes
from GenAIRR.pipeline import AugmentationPipeline
from GenAIRR.data import builtin_heavy_chain_data_config,builtin_lambda_chain_data_config,builtin_kappa_chain_data_config
from GenAIRR.steps import SimulateSequence,FixVPositionAfterTrimmingIndexAmbiguity,FixDPositionAfterTrimmingIndexAmbiguity,FixJPositionAfterTrimmingIndexAmbiguity
from GenAIRR.steps import CorrectForVEndCut,CorrectForDTrims,CorruptSequenceBeginning,InsertNs,InsertIndels,ShortDValidation,DistillMutationRate
from GenAIRR.mutation import S5F
from GenAIRR.steps.StepBase import AugmentationStep
from GenAIRR.pipeline import CHAIN_TYPE_BCR_HEAVY,CHAIN_TYPE_BCR_LIGHT_KAPPA,CHAIN_TYPE_BCR_LIGHT_LAMBDA
from GenAIRR.simulation import HeavyChainSequenceAugmentor, LightChainSequenceAugmentor, SequenceAugmentorArguments
from GenAIRR.utilities import DataConfig
from GenAIRR.data import builtin_heavy_chain_data_config,builtin_kappa_chain_data_config,builtin_lambda_chain_data_config
# Initialize DataConfig with the path to your configuration
#data_config = DataConfig('/path/to/your/config')
# Or Use one of Our Builtin Data Configs
data_config_builtin = builtin_heavy_chain_data_config()



## Simulating Heavy Chain Sequences

Let's simulate a BCR heavy chain sequence using the default GenAIRR pipeline for BCR heavy chain sequences via the `AugmentationPipeline`. This example demonstrates a simple simulation with default settings.


In [3]:
# Set the dataconfig and the chain type for the simulations
AugmentationStep.set_dataconfig(builtin_heavy_chain_data_config(),chain_type=CHAIN_TYPE_BCR_HEAVY)

# Create the simulation pipeline
pipeline = AugmentationPipeline([
    SimulateSequence(mutation_model = S5F(min_mutation_rate=0.003,max_mutation_rate=0.25),productive = True),
    FixVPositionAfterTrimmingIndexAmbiguity(),
    FixDPositionAfterTrimmingIndexAmbiguity(),
    FixJPositionAfterTrimmingIndexAmbiguity(),
    CorrectForVEndCut(),
    CorrectForDTrims(),
    CorruptSequenceBeginning(corruption_probability = 0.7,corrupt_events_proba = [0.4,0.4,0.2],max_sequence_length = 576,nucleotide_add_coefficient = 210,
                             nucleotide_remove_coefficient = 310,nucleotide_add_after_remove_coefficient = 50,random_sequence_add_proba = 1,
                             single_base_stream_proba = 0,duplicate_leading_proba = 0,random_allele_proba = 0),
    InsertNs(n_ratio = 0.02,proba = 0.5),
    ShortDValidation(short_d_length= 5),
    InsertIndels(indel_probability = 0.5,max_indels = 5,insertion_proba=0.5,deletion_proba=0.5),
    DistillMutationRate()
    ])



# Simulate a heavy chain sequence
heavy_sequence = pipeline.execute()

# Print the simulated heavy chain sequence
print("Simulated Heavy Chain Sequence:", heavy_sequence.get_dict())


Simulated Heavy Chain Sequence: {'sequence': 'CGNGNCCAGGTTATGCAGTCTGGGGCTCAGGTGACCAAGTATGGGGCCTCCGTAAAGGCTTTCTGTACGGCCTCTGGAGATACCTTCACTAGCCATACTATGCATTGGGNGCGCCTGGCCCCCGGACAGAGGCTTAAGGGAATGGGCTGGATTTAAAGTGACGATGGTTANACCGTCTGGTCACGGAGGCTACAGGNCAGAGTNACCCTTACCAGGGCCATGTTCGCGGGAAGAGCCTAGAGGGACTTCAGAAGCCTGAGATTCGAAGACACGGCTTTGNGTTAATGCGCGCGAGGTTGTGTTAAAATTCGGAGGTTTCCGGGCGGAGTTACTCTTGATGTCGGGGGCGAAGGAACCCTGGCACCGACTCCTCAGCCCTTCACCACGA', 'v_call': ['IGHVF6-G26*04'], 'd_call': ['IGHD3-10*01'], 'j_call': ['IGHJ3*02'], 'c_call': ['IGHG4*01'], 'v_sequence_start': 0, 'v_sequence_end': 295, 'd_sequence_start': 297, 'd_sequence_end': 317, 'j_sequence_start': 329, 'j_sequence_end': 375, 'v_germline_start': 0, 'v_germline_end': 295, 'd_germline_start': 3, 'd_germline_end': 23, 'j_germline_start': 3, 'j_germline_end': 50, 'junction_sequence_start': 285, 'junction_sequence_end': 345, 'mutation_rate': 0.23969072164948454, 'mutations': {1: 'A>G', 9: 'C>G', 12: 'G>A', 27: 'G>C', 34: 'A>C', 35: 'G>C', 39:

## Customizing Simulations

GenAIRR allows for extensive customization to closely mimic the natural diversity of immune sequences. Below is an example of how to customize mutation rates and indel simulations.


In [ ]:
# Customize augmentation arguments

custom_mutation_model = S5F(min_mutation_rate=0.1,max_mutation_rate=0.5)
custom_insert_indel_step = InsertIndels(indel_probability = 0.05,max_indels = 15,insertion_proba=0.7,deletion_proba=0.3)

pipeline = AugmentationPipeline([
    SimulateSequence(mutation_model = custom_mutation_model,productive = True), # notice here in the simulate sequence step we used the custom mutation model we defined
    FixVPositionAfterTrimmingIndexAmbiguity(),
    FixDPositionAfterTrimmingIndexAmbiguity(),
    FixJPositionAfterTrimmingIndexAmbiguity(),
    CorrectForVEndCut(),
    CorrectForDTrims(),
    CorruptSequenceBeginning(corruption_probability = 0.7,corrupt_events_proba = [0.4,0.4,0.2],max_sequence_length = 576,nucleotide_add_coefficient = 210,
                             nucleotide_remove_coefficient = 310,nucleotide_add_after_remove_coefficient = 50,random_sequence_add_proba = 1,
                             single_base_stream_proba = 0,duplicate_leading_proba = 0,random_allele_proba = 0),
    InsertNs(n_ratio = 0.02,proba = 0.5),
    ShortDValidation(short_d_length= 5),
    custom_insert_indel_step, # notice here we used the custom insert indel step we have created above
    DistillMutationRate()
    ])



# Simulate a heavy chain sequence
heavy_sequence = pipeline.execute()

# Print the simulated heavy chain sequence
print("Customized Simulated Heavy Chain Sequence:", heavy_sequence.get_dict())


Customized Simulated Heavy Chain Sequence: {'sequence': 'ACGACGTTACGTTGAATCTAGATGACCGCAGCCACGGCCGCCTCCTTGATAACGGCGGACATTGACGTGAGAAGAGTTAGATAAGGGGAAACGCCCCCTAGTCATTCTGCGCGCGAGCGACTTCTTATTATGGATATCCTCTTTGTCTCGCGGGGCAGGACGCACCTTTTCAACGTCTCGCCCGGCACACCTGAGCAGGGACAAGGTCCTT', 'v_call': ['IGHVF6-G21*11', 'IGHVF6-G21*09', 'IGHVF6-G21*07', 'IGHVF6-G21*06'], 'd_call': ['IGHD5-24*01'], 'j_call': ['IGHJ4*02'], 'c_call': ['IGHA2*02'], 'v_sequence_start': 16, 'v_sequence_end': 116, 'd_sequence_start': 127, 'd_sequence_end': 136, 'j_sequence_start': 140, 'j_sequence_end': 184, 'v_germline_start': 196, 'v_germline_end': 296, 'd_germline_start': 4, 'd_germline_end': 13, 'j_germline_start': 4, 'j_germline_end': 48, 'junction_sequence_start': 105, 'junction_sequence_end': 153, 'mutation_rate': 0.33649289099526064, 'mutations': {18: 'A>T', 19: 'G>A', 20: 'A>G', 21: 'G>A', 23: 'C>T>G', 26: 'G>C', 27: 'A>G', 28: 'T>C', 29: 'T>A', 30: 'A>G', 33: 'G>A', 37: 'A>G>C', 40: 'A>C', 41: 'A>C', 45: 'A>T', 46: 'C>G>T

## Generating Naïve Sequences

In immunogenetics, a naïve sequence refers to an antibody sequence that has not undergone the process of somatic hypermutation. GenAIRR allows you to simulate such naïve sequences using the `HeavyChainSequence` class. Let's start by generating a naïve heavy chain sequence.


In [52]:
from GenAIRR.sequence import HeavyChainSequence

# Create a naive heavy chain sequence
naive_heavy_sequence = HeavyChainSequence.create_random(data_config_builtin)

# Access the generated naive sequence
naive_sequence = naive_heavy_sequence

print("Naïve Heavy Chain Sequence:", naive_sequence)
print('Ungapped Sequence: ')
print(naive_sequence.ungapped_seq)


Naïve Heavy Chain Sequence: 0|-----------------------------------------------------------------------------V(IGHVF3-G8*01)|294|296|----D(IGHD2-8*02)|312|332|------------J(IGHJ2*01)|381
Ungapped Sequence: 
CAGGTGCAGCTGCAGGAGTCGGGCCCAGGACTGGTGAAGCCTCCGGGGACCCTGTCCCTCACCTGCGCTGTCTCTGGTGGCTCCATCAGCAGTAGTAACTGGTGGAGTTGGGTCCGCCAGCCCCCAGGGAAGGGGCTGGAGTGGATTGGGGAAATCTATCATAGTGGGAGCACCAACTACAACCCGTCCCTCAAGAGTCGAGTCACCATATCAGTAGACAAGTCCAAGAACCAGTTCTCCCTGAAGCTGAGCTCTGTGACCGCCGCGGACACGGCCGTGTATTGCTGTGCGAGAAAACTGGTGGTGTATGCTCCTATCTCCCACATAGGGCTTGGTACTTCGATCTCTGGGGCCGTGGCACCCTGGTCACTGTCTCCTCAG


## Applying Mutations

To mimic the natural diversity and evolution of immune sequences, GenAIRR supports the simulation of mutations through various models. Here, we demonstrate how to apply mutations to a naïve sequence using the `S5F` and `Uniform` mutation models from the mutations submodule.


### Using the S5F Mutation Model

The `S5F` model is a sophisticated mutation model that considers context-dependent mutation probabilities. It's particularly useful for simulating realistic somatic hypermutations.


In [53]:
from GenAIRR.mutation import S5F

# Initialize the S5F mutation model with custom mutation rates
s5f_model = S5F(min_mutation_rate=0.01, max_mutation_rate=0.05)

# Apply mutations to the naive sequence using the S5F model
s5f_mutated_sequence, mutations, mutation_rate = s5f_model.apply_mutation(naive_heavy_sequence)

print("S5F Mutated Heavy Chain Sequence:", s5f_mutated_sequence)
print("S5F Mutation Details:", mutations)
print("S5F Mutation Rate:", mutation_rate)


S5F Mutated Heavy Chain Sequence: CAGGTGCAGCTGCAGGAGTCGGGCCCAGGACTGGTGAAGCCTCCGGGGACCCTGTCCCTCACCTGCGCTGTCTCTGCTGGCTCCATCAGCAGTAGTAACTGGTGGAGTTGGGTCCGCCAGCCCCCAGGGAAGGGGCTGGAGTGGATTGGGGAAATCTATCATAGTGGGAGCACCAACTACAACCCGTCCCTCTAGAGTCGAGTCACCATATCAGTAGACAAGTCCAAGAACCAGTTCTCCCTGAAGCAGAGCTCTGTGACCGCCGCGGACTCGGCCGTGTATTGCTGTGCGAGAAAACTGGTGGTGTATGCTCCTATCTCCCACATAGGGCTTGGTACTTCGATCTCTGGGGCCGTGGCACCCTGGTCACTGTCTCCTCAG
S5F Mutation Details: {270: 'A>T', 192: 'A>T', 247: 'T>A', 76: 'G>C'}
S5F Mutation Rate: 0.011222406361310347


### Using the Uniform Mutation Model

The `Uniform` mutation model applies mutations at a uniform rate across the sequence, providing a simpler alternative to the context-dependent models.


In [54]:
from GenAIRR.mutation import Uniform

# Initialize the Uniform mutation model with custom mutation rates
uniform_model = Uniform(min_mutation_rate=0.01, max_mutation_rate=0.05)

# Apply mutations to the naive sequence using the Uniform model
uniform_mutated_sequence, mutations, mutation_rate = uniform_model.apply_mutation(naive_heavy_sequence)

print("Uniform Mutated Heavy Chain Sequence:", uniform_mutated_sequence)
print("Uniform Mutation Details:", mutations)
print("Uniform Mutation Rate:", mutation_rate)


Uniform Mutated Heavy Chain Sequence: CAGGTGCACCTGCAGGAGTCGGGCCGAGGAGTGGTGAAGCCTCCGGGGACCCTGTCCCTCACCTGCGCTGTCTCTGGTGGCTCCATCAGCAGTAGTTACTTGTGGAGTTGGGTCCGCCAGCCACCAGGGAAGGGGCTGGAGTGGATTGGGGAAATCTATCATAGTGGGAGCACCAACTACAACCCGTCCCTCAAGAGTCGAGTCACCATATCAGTAGACAAGTCCAAGAACCAGTTCTCCCTGAAGCTGAGCTCTGTGACCGCCGCGGACACGGCCGTGTATTGCTGTGCGAGAAAACTGGTGGTGTATGCTCCTATCTCCCACATAGGGCTTGGTACTTCGATCTGTGGGGCCGTGGCACCCTGGTCACTGTCTCCTCAG
Uniform Mutation Details: {122: 'C>A', 8: 'G>C', 100: 'G>T', 96: 'A>T', 25: 'C>G', 346: 'C>G', 30: 'C>G'}
Uniform Mutation Rate: 0.019802269687583134


## Common Use Cases

GenAIRR is a versatile tool designed to meet a broad range of needs in immunogenetics research. This section provides examples and explanations for some common use cases, including generating multiple sequences, simulating specific allele combinations, and more.


### Generating Many Sequences

One common requirement is to generate a large dataset of synthetic AIRR sequences for analysis or benchmarking. Below is an example of how to generate multiple sequences using GenAIRR in a loop.


In [55]:
num_sequences = 5  # Number of sequences to generate

heavy_sequences = []
for _ in range(num_sequences):
    # Simulate a heavy chain sequence
    heavy_sequence = heavy_augmentor.simulate_augmented_sequence()
    heavy_sequences.append(heavy_sequence)

# Display the generated sequences
for i, seq in enumerate(heavy_sequences, start=1):
    print(f"Heavy Chain Sequence {i}: {seq}")


Heavy Chain Sequence 1: {'sequence': 'TTGGNAAGCCAGGCCCTGGAGTGACTTTCACACACTGATCGGTGCGANGCTGAACTCCACAACGCCTCCCTCAAAACCTGACCCACCACGTCCAGGGACCCGTCCGGTAGTCACATGGTCCTGACACTGTCGAACATGGACCCTGTGGACACAGTCACACATTACTGTGCACCGATNCCCCCCCCTACGANGATTCCGGCCGGGCCCTGGCTAATCCAATCACTTGTTGGAGGTCTGGGGCAAAGGGACCACGGCCACCGACTCNTAAG', 'v_sequence_start': 9, 'v_sequence_end': 176, 'd_sequence_start': 185, 'd_sequence_end': 199, 'j_sequence_start': 207, 'j_sequence_end': 269, 'v_call': 'IGHVF1-G3*06,IGHVF1-G3*05,IGHVF1-G3*04', 'd_call': 'IGHD3-10*03', 'j_call': 'IGHJ6*03', 'mutation_rate': 0.241635687732342, 'v_trim_5': 0, 'v_trim_3': 2, 'd_trim_5': 4, 'd_trim_3': 13, 'j_trim_5': 2, 'j_trim_3': 0, 'corruption_event': 'remove_before_add', 'corruption_add_amount': 9, 'corruption_remove_amount': 132, 'indels': {}}
Heavy Chain Sequence 2: {'sequence': 'CAGCTGCAGTTGCAGGAGTCGGGCCCNGGACTGGTGAAGCCTTTGGAGGCCCAGTCCCTCTCGTACCCTGTCTCTGGTGACTCCATCAGCAATAGTGGTTACTCCTGGGGCTGAATCCGTCCCCCCNCAGGGAAGGGGCTGGAGTGGATNGCGACTATANATTATAGG

In [56]:
import pandas as pd
pd.DataFrame(heavy_sequences)

,sequence,v_sequence_start,v_sequence_end,d_sequence_start,d_sequence_end,j_sequence_start,j_sequence_end,v_call,d_call,j_call,...,v_trim_5,v_trim_3,d_trim_5,d_trim_3,j_trim_5,j_trim_3,corruption_event,corruption_add_amount,corruption_remove_amount,indels
0,TTGGNAAGCCAGGCCCTGGAGTGACTTTCACACACTGATCGGTGCG...,9,176,185,199,207,269,"IGHVF1-G3*06,IGHVF1-G3*05,IGHVF1-G3*04",IGHD3-10*03,IGHJ6*03,...,0,2,4,13,2,0,remove_before_add,9,132,{}
1,CAGCTGCAGTTGCAGGAGTCGGGCCCNGGACTGGTGAAGCCTTTGG...,0,297,308,317,320,370,IGHVF3-G10*06,IGHD3-9*01,IGHJ5*02,...,0,2,8,15,1,0,no-corruption,0,0,{}
2,CAGAAGAGACTGGTGCAGTCTGGGGTTGACATGAAGACGACTGGGT...,0,295,295,305,316,367,IGHVF6-G20*02,"IGHD4-11*01,IGHD4-4*01",IGHJ2*01,...,0,1,3,3,3,0,no-corruption,0,0,{}
3,CAGTTTCAGCTGGTGCCGTCTGGAGCTGAGGTGAAGAAGNCTGNGG...,0,294,316,323,332,391,IGHVF6-G25*02,"IGHD5-18*01,IGHD5-5*01",IGHJ6*02,...,0,2,8,6,4,0,no-corruption,0,0,{}
4,GAGGTGCAACTGCTGCAGACCGGGTCAGACTTGATACAGCCAGGGG...,0,296,302,306,310,370,IGHVF10-G41*02,Short-D,IGHJ6*03,...,0,0,7,6,4,0,no-corruption,0,0,{}


### Generating a Specific Allele Combination Sequence

In some cases, you might want to simulate sequences with specific V, D, and J allele combinations. Here's how to specify alleles for your simulations.


In [57]:
# Define your specific alleles
v_allele = 'IGHVF6-G21*01'
d_allele = 'IGHD5-18*01'
j_allele = 'IGHJ6*03'

# Extract the allele objects from data_config
v_allele = next((allele for family in data_config_builtin.v_alleles.values() for allele in family if allele.name == v_allele), None)
d_allele = next((allele for family in data_config_builtin.d_alleles.values() for allele in family if allele.name == d_allele), None)
j_allele = next((allele for family in data_config_builtin.j_alleles.values() for allele in family if allele.name == j_allele), None)

# Check if all alleles were found
if not v_allele or not d_allele or not j_allele:
    raise ValueError("One or more specified alleles could not be found in the data config.")


# Generate a sequence with the specified allele combination
specific_allele_sequence = HeavyChainSequence([v_allele, d_allele, j_allele], data_config_builtin)
specific_allele_sequence.mutate(s5f_model)



print("Specific Allele Combination Sequence:", specific_allele_sequence.mutated_seq)


Specific Allele Combination Sequence: CAGGTGCAGTTGGTGCAGTCTGGGACTGAGTTGAAGACGCCTGGGTCCTCGGTGAAGGTCTCCTGCAAGGCTTCTAGAGGCACCTTCAGCAGCTCTGCTATCAGCTGGGTGCGACAGGCCCCTGGACAAGGGCTTGAGTGGATGGGAGGGATCATCCCTATCTTTGGTACAGCAAACTACGCACAGAAGTTCCAGGGCAGAGTCACGATTACCGCGGATAAATCCACGAGCACAGCCTACATGGAGCTGAGCAGCCTGAGATCTGAGGATACGGCCGTGTATTACTGTGCGAGAGAGGATGGGTCCGGATCCCACCCCATTTACTATTACTACTACTACATGGACGTCTGGGGCAAAGGGACCACGGTCACCGTCTCCTCAG


### Generating Naïve vs. Mutated Sequence Pairs

Comparing naïve and mutated versions of the same sequence can be useful for studying somatic hypermutation effects. Here's how to generate such pairs with GenAIRR.


In [59]:
# Generate a naive sequence
sequence_object = HeavyChainSequence.create_random(data_config_builtin)
sequence_object.mutate(s5f_model)

print("Naïve Sequence:", sequence_object.ungapped_seq)
print("Mutated Sequence:", sequence_object.mutated_seq)


Naïve Sequence: CAGGTGCAGCTGCAGGAGTCGGGCCCAGGACTGGTGAAGCCTTCGGACACCCTGTCCCTCACCTGCGCTGTCTCTGGTTACTCCATCAGCAGTAGTAACTGGTGGGGCTGGATCCGGCAGCCCCCAGGGAAGGGACTGGAGTGGATTGGGTACATCTATTATAGTGGGAGCATCTACTACAACCCGTCCCTCAAGAGTCGAGTCACCATGTCAGTAGACACGTCCAAGAACCAGTTCTCCCTGAAGCTGAGCTCTGTGACCGCCGTGGACACGGCCGTGTATTACTGTGCGAGAAAGCCACTCGGTCACACTACGGTGGTAACTCATGCTTTTGATATCTGGGGCCAAGGGACAATGGTCACCGTCTCTTCAG
Mutated Sequence: CAGGTGCAGCTGCAGGAGTCGGGCCCAGGACTGGTGAAGCCTTCGGACACCCTGTCCCTCACCTGCGCTGTCTCTGGTTACTCCATCAGCAGTAGTAACTGGTGGGGCTGGATCCGGCAGCCCCCAGGGAAGGGACTGGAGTGGATTGGGAACATCCATTATAGTGGGAGCATCTACTACAACCCGTCCCTCAAGAGTCGAGTCACCATGTCACTAGACACGTCCAAGAACCAGTTCTCCCTGAAACTGAGCTCTGTGGCCGCCGTGGACACGGCCGTGTATTACTGTGCGAGAACGCCACTCGGTCACACTACGGTGGTAATTCATGCTTTTGATATCTGGGGCCAAGGGACAATGGTCACCGTCTCTTCAG


## Conclusion

This section highlighted some common use cases for GenAIRR, demonstrating its flexibility in simulating AIRR sequences for various research purposes. Whether you need large datasets, specific allele combinations, custom mutation rates, or comparative analyses of naïve and mutated sequences, GenAIRR provides the necessary tools to achieve your objectives.
